In [2]:
import pandas as pd
import os
from unidecode import unidecode
import warnings
warnings.filterwarnings('ignore')

# Definir rutas de los archivos:

In [4]:
project_directory = os.path.abspath('../')
vehiculos_directory = os.path.join(project_directory, 'data', 'antiguedad_vehiculos_y_combustible')
vehiculos_files = os.listdir(vehiculos_directory)

# Exploración archivos:

## Reducción de columnas:

Para optimizar el uso del espacio en el bucket de data refinada en S3 y procesar solo la data necesaria en Lambda, se procede a identificar las columnas que no serán utilizadas.

### Definir campos comunes entre los archivos:

La lista resultante de campos será la primera base para definir los campos a guardar en el bucket de data refinada.

In [5]:
ls_raw_fields = []
for file in vehiculos_files:
    file_path = os.path.join(vehiculos_directory, file)
    df = pd.read_excel(file_path,nrows=0)
    ls_raw_fields_aux = df.columns.to_list()
    if len(ls_raw_fields)==0:
        ls_raw_fields = ls_raw_fields_aux
    else:
        list(set(ls_raw_fields).intersection(ls_raw_fields_aux))
ls_raw_fields

['ANOMES',
 'CONFIGURACION',
 'RANGOMODELO',
 'COMBUSTIBLE',
 'PLACAS',
 'VIAJES',
 'VIAJESVACIOS',
 'VALORPACTADO',
 'KILOGRAMOS',
 'GALONES',
 'VIAJESVALORCERO',
 'VIAJESLIQUIDOS']

### Definir campos finales:

Estos serán los campos a consultar, explorar y guardar en el bucket de data refinada.

In [6]:
ls_refined_fields = ls_raw_fields
print(len(ls_refined_fields))

12


## Identificar y definir tipos de datos:

El objetivo es identificar el tipo de datos presente en todos los archivos y saber si corresponden entre archivo y archivo. Estos serán el tipo de campos a asegurar durante el proceso de limpieza.

In [7]:
ls_raw_fields = []
df_info = pd.DataFrame()
for file in vehiculos_files:
    file_path = os.path.join(vehiculos_directory, file)
    df = pd.read_excel(file_path,usecols=ls_refined_fields)
    ls_columns = df.columns.tolist()
    ls_data_types = df.dtypes.tolist()
    df_info_aux = pd.DataFrame({'column_name': ls_columns, 'data_type': ls_data_types})
    if df_info.empty:
        df_info = df_info_aux
    else:
        df_info = pd.merge(df_info,df_info_aux,on = ['column_name','data_type'],how='left')

val =  len(df_info) == len(ls_refined_fields)
print(df_info)
print(val)

        column_name data_type
0            ANOMES     int64
1     CONFIGURACION    object
2       RANGOMODELO    object
3       COMBUSTIBLE    object
4            PLACAS     int64
5            VIAJES     int64
6      VIAJESVACIOS     int64
7      VALORPACTADO     int64
8        KILOGRAMOS     int64
9           GALONES     int64
10  VIAJESVALORCERO     int64
11   VIAJESLIQUIDOS     int64
True


### Se crea un diccionario con el tipo de datos:

Este diccionario se usará en el proceso de lectra de los archivos para asegurar el formato de los campos.

In [8]:
dict_types = df_info.set_index('column_name')['data_type'].to_dict()
# df = pd.read_excel(file_path,nrows=100,usecols=ls_refined_fields,dtype=dict_types)
dict_types

{'ANOMES': dtype('int64'),
 'CONFIGURACION': dtype('O'),
 'RANGOMODELO': dtype('O'),
 'COMBUSTIBLE': dtype('O'),
 'PLACAS': dtype('int64'),
 'VIAJES': dtype('int64'),
 'VIAJESVACIOS': dtype('int64'),
 'VALORPACTADO': dtype('int64'),
 'KILOGRAMOS': dtype('int64'),
 'GALONES': dtype('int64'),
 'VIAJESVALORCERO': dtype('int64'),
 'VIAJESLIQUIDOS': dtype('int64')}

## Validaciones:

En este paso se establecen las validaciones de los archivos, en caso de que algún archivo no supere la fase de validación no será cargado en el bucket de data refinada.

### Validar formato de archivos:

Se valida si el formato de los archivos

In [9]:
for file in vehiculos_files:
    if not file_path.endswith('.xlsx'):
        val = False
    else:
        val = True
    print(f'file: {file} | validation: {val}')

file: EstadisticasRNDC_202100Placas.xlsx | validation: True


###  Validar campos:

Se valida si todos los campos definidos están presentes en el archivo.

In [10]:
for file in vehiculos_files:
    file_path = os.path.join(vehiculos_directory, file)
    df = pd.read_excel(file_path,nrows=0)
    val = all(elem in df.columns.to_list() for elem in ls_refined_fields)
    print(f'file {file} | validation: {val}')

file EstadisticasRNDC_202100Placas.xlsx | validation: True


### Validar coherencia en los datos:

Se valida si el periodo al que hace referencia el nombre del archivo corresponde con el dato de la columna "MES". Para el momento de la limpieza si el archivo no cumple esta validación no será procesado ni cargado en el bucked de data refinada.

In [22]:
for file in vehiculos_files:
    file_path = os.path.join(vehiculos_directory, file)
    period = float(file.split('_')[-1].split('.')[0][:6])
    period = period
    df = pd.read_excel(file_path,usecols=['ANOMES'])
    period_avg = df.ANOMES.mean()
    val = period == period_avg
    print(f'file: {file} | period: {period} | period_avg: {period_avg} | validation: {val}')

file: EstadisticasRNDC_202100Placas.xlsx | period: 202100.0 | period_avg: 202100.0 | validation: True


# Transformaciones y limpieza:

En este paso entrarán todos los archivos que hayan pasado la fase de validación. Sin embargo, para el ejemplo de este notebook ssolo se usará el archivo "EstadisticasRNDC_202207.xlsx"

In [25]:
file = 'EstadisticasRNDC_202100Placas.xlsx'
file_path = os.path.join(vehiculos_directory, file)
df = pd.read_excel(file_path,usecols=ls_refined_fields,dtype=dict_types)
df.rename(columns={'CONFIGURACION':'COD_CONFIG_VEHICULO'},inplace=True)
df.drop_duplicates(inplace=True) # eliminar duplicados
df = df.applymap(lambda x: x.upper() if isinstance(x, str) else x) # volver mayúsculas los campos str
df = df.applymap(lambda x: unidecode(x) if isinstance(x, str) else x) # quitar acentos a campos str
print(f'file: {file} | status: refined')


file: EstadisticasRNDC_202100Placas.xlsx | status: refined


In [26]:
df.head()

,ANOMES,COD_CONFIG_VEHICULO,RANGOMODELO,COMBUSTIBLE,PLACAS,VIAJES,VIAJESVACIOS,VALORPACTADO,KILOGRAMOS,GALONES,VIAJESVALORCERO,VIAJESLIQUIDOS
0,202100,4R4,1991-1995,DIESEL,1,1,0,4100000,25198,0,0,0
1,202100,3R3,1950-1970,DIESEL,1,2,0,3750000,40000,0,0,0
2,202100,3S1,2011-2015,BIODIESEL,1,5,0,8600000,71000,0,0,0
3,202100,3S2S2,1996-2000,DIESEL,1,1,0,1000000,10000,0,0,0
4,202100,3S2,1981-1985,GAS,1,4,0,0,80000,0,4,0
